In [ ]:
%pip install tabulate

In [1]:
import mysql.connector
from mysql.connector import Error

mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",
 database = "choco_crunch"
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

In [2]:
import pandas as pd

--PRODUCT_INFO QUERIES--

In [ ]:
#1. Count products per brand
mycursor.execute('''SELECT brand, COUNT(*) AS product_count 
                 FROM product_info GROUP BY brand ORDER BY product_count DESC''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#2. Count unique products per brand
mycursor.execute('''SELECT brand, COUNT(DISTINCT product_name) AS unique_products 
                 FROM product_info GROUP BY brand ORDER BY unique_products DESC''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#3. Top 5 brands by product count
mycursor.execute('''SELECT brand, COUNT(*) AS product_count 
                 FROM product_info GROUP BY brand ORDER BY product_count DESC LIMIT 5''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#4. Products with missing product name
mycursor.execute('''SELECT * FROM product_info 
                 WHERE product_name IS NULL OR product_name = '' OR product_name = 'Unknown Product' ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#5. Number of unique brands
mycursor.execute("SELECT COUNT(DISTINCT brand) AS total_unique_brands FROM product_info")

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#6. Products with code starting with '3'
mycursor.execute("SELECT * FROM product_info WHERE product_code LIKE '3%'")

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

-- NUTRIENT_INFO QUERIES --

In [ ]:
#1. Top 10 products with highest energy_kcal_value
mycursor.execute('''SELECT product_code, energy_kcal_value 
                 FROM nutrient_info ORDER BY energy_kcal_value DESC LIMIT 10''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#2. Average sugars_value per nova_group
mycursor.execute('''SELECT nova_group, AVG(sugars_value) AS avg_sugar 
                 FROM nutrient_info GROUP BY nova_group ORDER BY nova_group''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#3. Count products with fat_value > 20g
mycursor.execute("SELECT COUNT(*) AS high_fat_products FROM nutrient_info WHERE fat_value > 20")

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#4. Average carbohydrates_value per product
mycursor.execute("SELECT AVG(carbohydrates_value) AS avg_carbs FROM nutrient_info")

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#5. Products with sodium_value > 1g
mycursor.execute("SELECT product_code, sodium_value FROM nutrient_info WHERE sodium_value > 1")

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#6. Count products with non-zero fruits/vegetables/nuts content
mycursor.execute('''SELECT COUNT(*) AS products_with_fruits FROM nutrient_info 
                 WHERE fruits_vegetables_nuts_estimate_from_ingredients_100g > 0''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#7. Products with energy_kcal_value > 500
mycursor.execute('''SELECT product_code, energy_kcal_value 
                 FROM nutrient_info WHERE energy_kcal_value > 500
                 ORDER BY energy_kcal_value DESC''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

-- DERIVED_METRICS QUERIES --

In [ ]:
#1. Count products per calorie_category
mycursor.execute('''SELECT calorie_category, COUNT(*) AS product_count 
                 FROM derived_metrics GROUP BY calorie_category''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#2. Count of High Sugar products
mycursor.execute('''SELECT COUNT(*) AS high_sugar_products 
                 FROM derived_metrics WHERE sugar_category = 'High Sugar' ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#3. Average sugar_to_carb_ratio for High Calorie products
mycursor.execute('''SELECT AVG(sugar_to_carb_ratio) AS avg_ratio_high_calorie 
                 FROM derived_metrics WHERE calorie_category = 'High' ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#4. Products that are both High Calorie and High Sugar
mycursor.execute('''SELECT * FROM derived_metrics 
                 WHERE calorie_category = 'High' AND sugar_category = 'High Sugar' ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#5. Number of ultra-processed products
mycursor.execute('''SELECT COUNT(*) AS ultra_processed_count 
                 FROM derived_metrics WHERE is_ultra_processed = 'Yes' ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#6. Products with sugar_to_carb_ratio > 0.7
mycursor.execute('''SELECT * FROM derived_metrics 
                 WHERE sugar_to_carb_ratio > 0.7 ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#7. Average sugar_to_carb_ratio per calorie_category
mycursor.execute('''SELECT calorie_category, AVG(sugar_to_carb_ratio) AS avg_ratio 
                 FROM derived_metrics GROUP BY calorie_category ORDER BY avg_ratio DESC''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

-- JOIN QUERIES (Multi-Table Analysis) --

In [ ]:
#1. Top 5 brands with most High Calorie products
mycursor.execute('''SELECT p.brand, COUNT(*) AS high_calorie_count 
                 FROM product_info p JOIN derived_metrics d ON p.product_code = d.product_code 
                 WHERE d.calorie_category = 'High' GROUP BY p.brand 
                 ORDER BY high_calorie_count DESC LIMIT 5''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#2. Average energy_kcal_value for each calorie_category
mycursor.execute('''SELECT d.calorie_category, 
                 AVG(n.energy_kcal_value) AS avg_energy 
                 FROM derived_metrics d JOIN nutrient_info 
                 n ON d.product_code = n.product_code 
                 GROUP BY d.calorie_category ORDER BY avg_energy DESC''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#3. Count of ultra-processed products per brand
mycursor.execute('''SELECT p.brand, COUNT(*) AS ultra_processed_count 
                 FROM product_info p JOIN derived_metrics d 
                 ON p.product_code = d.product_code 
                 WHERE d.is_ultra_processed = 'Yes' 
                 GROUP BY p.brand ORDER BY ultra_processed_count DESC ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#4. Products with High Sugar and High Calorie along with brand
mycursor.execute('''SELECT p.brand, p.product_name, d.sugar_category, d.calorie_category 
                 FROM product_info p JOIN derived_metrics d 
                 ON p.product_code = d.product_code 
                 WHERE d.sugar_category = 'High Sugar' AND d.calorie_category = 'High' ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#5. Average sugar content per brand for ultra-processed products
mycursor.execute('''SELECT p.brand, AVG(n.sugars_value) 
                 AS avg_sugar_ultra FROM product_info p 
                 JOIN nutrient_info n ON p.product_code = n.product_code 
                 JOIN derived_metrics d ON p.product_code = d.product_code 
                 WHERE d.is_ultra_processed = 'Yes' GROUP BY p.brand 
                 ORDER BY avg_sugar_ultra DESC LIMIT 10''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#6. Number of products with fruits/vegetables/nuts content in each calorie_category
mycursor.execute('''SELECT d.calorie_category, COUNT(*) AS fruit_nut_products 
                 FROM nutrient_info n JOIN derived_metrics d 
                 ON n.product_code = d.product_code 
                 WHERE n.fruits_vegetables_nuts_estimate_from_ingredients_100g > 0 
                 GROUP BY d.calorie_category''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)

In [ ]:
#7. Top 5 products by sugar_to_carb_ratio with their calorie and sugar category
mycursor.execute('''SELECT p.product_name, d.sugar_to_carb_ratio, d.calorie_category, d.sugar_category 
                 FROM product_info p 
                 JOIN derived_metrics d ON p.product_code = d.product_code 
                 ORDER BY d.sugar_to_carb_ratio DESC LIMIT 5 ''')

result = mycursor.fetchall()
df = pd.DataFrame(result, columns=[i[0] for i in mycursor.description])
display(df)